In [1]:
import os
import gc
import sys
import glob
import math
import numpy as np
import uproot
import pandas
from functools import partial
from concurrent.futures import ThreadPoolExecutor

import keras
import tensorflow as tf
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Conv2D, Dropout, AlphaDropout, Activation, BatchNormalization, Flatten
from keras.layers.merge import Concatenate
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras_tqdm import TQDMNotebookCallback

sys.path.insert(0, "../../python")
from common import *
from DataLoader import DataLoader

C:\Users\konst\AppData\Roaming\Python\Python36\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def conv_block(prev_layer, filters, kernel_size, dropout_rate, block_name, n):
    conv = Conv2D(filters, kernel_size, name="{}_conv_{}".format(block_name, n),
                  kernel_initializer='lecun_normal', activation='selu')(prev_layer)
    if dropout_rate > 0:
        return AlphaDropout(dropout_rate, name="{}_dropout_{}".format(block_name, n))(conv)
    return conv


def reduce_n_features_2d(input_layer, first_layer_size, last_layer_size, decay_factor, block_name, dropout_rate):
    conv_kernel=(1, 1)
    prev_layer = input_layer
    current_size = first_layer_size
    n = 1
    while True:
        prev_layer = conv_block(prev_layer, current_size, conv_kernel, dropout_rate, block_name, n)
        if current_size == last_layer_size: break
        current_size = max(last_layer_size, int(current_size / decay_factor))
        n += 1
    return prev_layer

def create_model():
    dense_dropout_rate = 0.25
    conv_dropout_rate = 0.25
    first_layer_size = 256
    last_layer_size = 16
    decay_factor = 2
    conv_decay_factor = 2
        
    model_name = "DeepTau2017v2p2"
    input_layer_tau = Input(name="input_tau", shape=(len(input_tau_branches),))
    input_layers = [ input_layer_tau ]
    
    high_level_features = [ input_layer_tau ]
    for loc in cell_locations:
        reduced_inputs = []
        for comp_id in range(len(component_names)):
            comp_name = component_names[comp_id]
            comp_branches = component_branches[comp_id]
            input_layer_comp = Input(name="input_{}_{}".format(loc, comp_name),
                                     shape=(n_cells_eta[loc], n_cells_phi[loc], len(comp_branches)))
            input_layers.append(input_layer_comp)
            reduced_comp = reduce_n_features_2d(input_layer_comp, 128, 16, conv_decay_factor, "{}_{}".format(loc, comp_name),
                                                conv_dropout_rate)
            reduced_inputs.append(reduced_comp)
        conv_all_start = Concatenate(name="{}_cell_concat".format(loc), axis=3)(reduced_inputs)
        cell_output_size = 32
        prev_layer = conv_block(conv_all_start, cell_output_size, (1, 1), conv_dropout_rate,
                                "{}_all".format(loc), 1)
        window_size = 4
        current_size = n_cells_eta[loc]
        n = 1
        while current_size > 1:
            win_size = min(current_size, window_size)
            prev_layer = conv_block(prev_layer, cell_output_size, (win_size, win_size), conv_dropout_rate,
                                    "{}_all_{}x{}".format(loc, win_size, win_size), n)
            n += 1
            current_size -= window_size - 1
            
        cells_flatten = Flatten(name="{}_cells_flatten".format(loc))(prev_layer)
        high_level_features.append(cells_flatten)
        
    prev_layer = Concatenate(name="features_concat")(high_level_features)
    current_size = first_layer_size
    n = 1
    while True:
        dense_layer = Dense(current_size, name="dense_%d" % n, kernel_initializer='lecun_normal', activation='selu')(prev_layer)
        if dense_dropout_rate > 0:
            prev_layer = AlphaDropout(dense_dropout_rate, name="dropout_%d" % n)(dense_layer)
        else:
            prev_layer = dense_layer
        n += 1
        if current_size == last_layer_size: break
        current_size = max(last_layer_size, int(current_size / decay_factor))

    output_layer = Dense(n_outputs, name="dense_%d" % n)(prev_layer)
    softmax_output = Activation("softmax", name="main_output")(output_layer)

    model = Model(input_layers, softmax_output, name="DeepTau2017v2")
    return model, model_name

In [3]:
def compile_model(model, learning_rate):
    #opt = keras.optimizers.Adam(lr=learning_rate)
    opt = keras.optimizers.Nadam(lr=learning_rate)
    #opt = keras.optimizers.SGD(lr=learning_rate, momentum=0.9, nesterov=True)
    #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
    metrics = ["accuracy", TauLosses.Le, TauLosses.Lmu, TauLosses.Ljet, TauLosses.sLe, TauLosses.sLmu, TauLosses.sLjet ]
    model.compile(loss=TauLosses.tau_crossentropy, optimizer=opt, metrics=metrics, weighted_metrics=metrics)

In [4]:
TauLosses.SetSFs(2, 1, 5)
model, model_name = create_model()
compile_model(model, 1e-3)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_inner_pfCand (InputLayer) (None, 21, 21, 36)   0                                            
__________________________________________________________________________________________________
input_inner_ele (InputLayer)    (None, 21, 21, 39)   0                                            
__________________________________________________________________________________________________
input_inner_muon (InputLayer)   (None, 21, 21, 39)   0                                            
__________________________________________________________________________________________________
inner_pfCand_conv_1 (Conv2D)    (None, 21, 21, 128)  4736        input_inner_pfCand[0][0]         
__________________________________________________________________________________________________
inner_ele_

__________________________________________________________________________________________________
outer_muon_conv_2 (Conv2D)      (None, 13, 13, 64)   8256        outer_muon_dropout_1[0][0]       
__________________________________________________________________________________________________
inner_all_conv_1 (Conv2D)       (None, 21, 21, 32)   1568        inner_cell_concat[0][0]          
__________________________________________________________________________________________________
outer_pfCand_dropout_2 (AlphaDr (None, 13, 13, 64)   0           outer_pfCand_conv_2[0][0]        
__________________________________________________________________________________________________
outer_ele_dropout_2 (AlphaDropo (None, 13, 13, 64)   0           outer_ele_conv_2[0][0]           
__________________________________________________________________________________________________
outer_muon_dropout_2 (AlphaDrop (None, 13, 13, 64)   0           outer_muon_conv_2[0][0]          
__________

__________________________________________________________________________________________________
outer_all_4x4_conv_4 (Conv2D)   (None, 1, 1, 32)     16416       outer_all_4x4_dropout_3[0][0]    
__________________________________________________________________________________________________
inner_all_3x3_dropout_7 (AlphaD (None, 1, 1, 32)     0           inner_all_3x3_conv_7[0][0]       
__________________________________________________________________________________________________
outer_all_4x4_dropout_4 (AlphaD (None, 1, 1, 32)     0           outer_all_4x4_conv_4[0][0]       
__________________________________________________________________________________________________
input_tau (InputLayer)          (None, 44)           0                                            
__________________________________________________________________________________________________
inner_cells_flatten (Flatten)   (None, 32)           0           inner_all_3x3_dropout_7[0][0]    
__________

In [5]:
def close_file(f_name):
    file_objs = [ obj for obj in gc.get_objects() if ("TextIOWrapper" in str(type(obj))) and (obj.name == f_name)]
    for obj in file_objs:
        obj.close()

In [6]:
def run_training(train_suffix, model_name, data_loader, epoch, n_epochs):

    train_name = '%s_%s' % (model_name, train_suffix)
    
    cb_acc = []
    for acc_name in ["acc", "weighted_acc"]:
        cb_acc.append(ModelCheckpoint("%s_acc.hdf5" % train_name, monitor="val_%s" % acc_name, save_best_only=True,
                                      save_weights_only=False, mode="max", verbose=1))
    
    cb_losses = []
    for loss_name in ["loss", "Le", "Lmu", "Ljet", "weighted_Le", "weighted_Lmu", "weighted_Ljet"]:
        cb_losses.append(ModelCheckpoint("%s_%s.hdf5" % (train_name, loss_name), monitor="val_%s" % loss_name,
                                         save_best_only=True, save_weights_only=False, mode="min", verbose=1))

    log_name = "%s.log" % train_name
    if os.path.isfile(log_name):
        close_file(log_name)
        os.remove(log_name)
    csv_log = CSVLogger(log_name, append=True)

    pbar = TQDMNotebookCallback(leave_outer=True, show_outer=True, leave_inner = True)
    callbacks = [pbar, csv_log, *cb_acc, *cb_losses]
    fit_hist = model.fit_generator(data_loader.generator(True), validation_data=data_loader.generator(False),
                                   steps_per_epoch=data_loader.steps_per_epoch, validation_steps=data_loader.validation_steps,
                                   callbacks=callbacks, epochs=n_epochs, initial_epoch=epoch, verbose=0)

    model.save("%s_final.hdf5" % train_name)
    return fit_hist

In [7]:
loader = DataLoader('file://N:/tau-ml/tuples-v2-t2/training.root', 500, 1000, validation_size=7491602,
                    max_queue_size=4, n_passes=-1)
print(loader.file_size, loader.data_size, loader.validation_size)

72491602 65000000 7491602


In [ ]:
fit_hist = run_training('step1', model_name, loader, 0, 4)